In [1]:
import random
from collections import *
from math import *
import pandas as pd
# import modin.pandas as pd
pd.set_option('display.max_columns', 500)
import numexpr, bottleneck
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from plotnine import *
import time

random.seed(a=1, version=2)

In [2]:
class Agent:
    
    def __init__(self, id, type=0, start_actions=(random.randint(0,2),random.randint(0,2)), 
                 coop_threshold=0.5, punish_threshold_enforcer=0.5, punish_threshold_conformist=0.5,
                 punish_threshold_changer=0.5, err_rate=0.05):
        self.id = id
        self.type = type
        self.actions = start_actions
        self.coop, self.punish  = self.actions 
        self.ct = coop_threshold
        self.pt_enforcer = punish_threshold_enforcer
        self.pt_conformist = punish_threshold_conformist
        self.pt_changer = punish_threshold_changer
        self.err_rate = err_rate
        
        
    def initiate_belief(self, b0='random'):
        if b0 == 'random':
            self.belief_coop  = random.random()
            self.belief_punish = random.random()
        elif b0 == 'good':
            self.belief_coop  = 1
            self.belief_punish = 1
        elif b0 == 'relatively good':
            self.belief_coop  = 0.75
            self.belief_punish = 0.75
        elif b0 == 'relatively bad':
            self.belief_coop  = 0.25
            self.belief_punish = 0.25
        elif b0 == 'bad':
            self.belief_coop  = 0
            self.belief_punish = 0
            
   
    def belief(self, sample):
        num_agents = len(sample)
        coops = [i.coop for i in sample]
        self.belief_coop = sum(coops) / num_agents
        punish = [i.punish for i in sample]
        self.belief_punish = sum(punish) / num_agents
            
    def choose_coop(self):
        # choose cooperation action
        if random.random() < self.err_rate: # with prob. = err_rate, randomly pick a cooperation action
            self.coop = random.randint(0,1)
        else:
            self.coop = self.belief_punish >= self.ct
        
    def choose_punish(self):
        # choose punishment action
        if random.random() < self.err_rate: # with prob. = err_rate, randomly pick a cooperation action
            self.punish = random.randint(0,1)
        else:
            # the never-punish type
            if self.type == 0: 
                self.punish = 0
            # independent punishers
            elif self.type == 1: 
                self.punish = 1
            # norm-enforcers who punish iff enough others cooperate
            elif self.type == 2: 
                self.punish = self.belief_coop > self.pt_enforcer
            # conformists who punish iff enough others punish
            elif self.type == 3: 
                self.punish = self.belief_punish > self.pt_conformist
            # norm changer who punish iff cooperation is rare
            elif self.type == 4: 
                self.punish = self.belief_coop < self.pt_changer
            

    def response(self):
        self.choose_coop()
        self.choose_punish()
    
    def update(self, sample, action='both'):
        self.belief(sample)
        if action=='both':
            self.choose_coop()
            self.choose_punish()
        elif action=='coop':
            self.choose_coop()
        elif action=='punish':
            self.choose_punish()
            
        
        
def summary(agents, belief=False):
    
    num = len(agents)
    cooperators = [i.coop for i in agents]
    prop_coop = sum(cooperators) / num
    
    punishers = [i.punish for i in agents]
    prop_punish = sum(punishers) / num
    
    if belief:
        beliefs_coop = [agents[i].belief_coop for i in range(n)]
        belief_coop = sum(beliefs_coop) / num

        beliefs_punish = [agents[i].belief_punish for i in range(n)]
        belief_punish = sum(beliefs_punish) / num
    
        summary = {"belief_coop_prop": belief_coop,
                   "belief_punish_prop": belief_punish,
                   "cooperate_prop": prop_coop,
                   "punish_prop": prop_punish
                  }
    else:
        summary = {"cooperate_prop": prop_coop,
                   "punish_prop": prop_punish}

    return summary



In [3]:

def run(n=100, T=100, types=(0,0,0), update_rate=0.5, e=0.1, sample_size=0.1, start_coop=0.5, start_punish=0.5,
       start_belief='random', start_by='belief'):
    
    
    # initial state
    
    type1,type2,type3,type4 = types
    types = [1]*ceil(type1*n/100) + [2]*ceil(type2*n/100) + [3]*ceil(type3*n/100) + [4]*ceil(type4*n/100) 
    types = types + [0] * (n - len(types))

    # set up agents
    if start_by == 'belief':
        start_state = zip(range(n), types)
        agents = [ Agent(id=i,type=t,err_rate=e) for i,t in start_state ]
        [ agents[i].initiate_belief(b0=start_belief) for i in range(n)]
        [ agents[i].response() for i in range(n) ]
    else:
        cooperation = [1] * round(start_coop*n) + [0] * (n - round(start_coop*n))
        punishment = [1] * round(start_punish*n) + [0] * (n - round(start_punish*n))
        start_state = zip(range(n), types, cooperation, punishment)
        agents = [ Agent(id=i,type=t,err_rate=e,start_actions=(c, p) ) for i,t,c,p in start_state ]
        
    
    # update for T rounds 

    cooperate_prop = []
    punish_prop = []
    
    if update_rate == 1:
        
        for t in range(T):
            state = summary(agents)
            cooperate_prop.append(state["cooperate_prop"])
            punish_prop.append(state["punish_prop"])
            
            last_round_agents = agents
            for i in range(n):
                sample = random.choices(last_round_agents, k=ceil(sample_size*n) )
                agents[i].update(sample)
            
    else:
        
        for t in range(T):
            state = summary(agents)
            cooperate_prop.append(state["cooperate_prop"])
            punish_prop.append(state["punish_prop"])

            last_round_agents = agents
            for i in range(n):
                if random.random() <= update_rate:
                    sample = random.choices(last_round_agents, k=ceil(sample_size*n) )
                    agents[i].update(sample)
                
    data = {"period": range(0, T),
            "fc_cooperate": cooperate_prop, # proportion of cooperators
            "fc_punish": punish_prop} # proportion of punishers
    
#     data = {"population_size": [n]*T,
#             "period": range(0, T),
#             "fc_cooperate": cooperate_prop, # proportion of cooperators
#             "fc_punish": punish_prop} # proportion of punishers
    
    return pd.DataFrame.from_dict(data)

     
def gen_type_list(t1=30, step=10, conditionals=[]):
    # proportion of each type 
    # t1 independent punishers
    # t2 norm-enforcers
    # t3 conformist punisher
    # t4 norm changer 
    types_list0 = []
    types_list1 = []
    if len(conditionals) > 0:
        for total in conditionals:
            for t2 in range(0,total,step):
                for t3 in range(0,total-t2+1,step):
                    t4 = total - t2 - t3
                    types_list0.append( (t1,t2,t3,t4) )
                for t4 in range(0,total-t2+1,step):
                    t3 = total - t2 - t4
                    types_list1.append( (t1,t2,t3,t4) )
    else:
        for t2 in range(0,101-t1,step):
            for t3 in range(0,101-t1-t2,step):
                for t4 in range(0,101-t1-t2-t3,step):
                    types_list0.append( (t1,t2,t3,t4) )
            for t4 in range(0,101-t1-t2,step):
                for t3 in range(0,101-t1-t2-t4,step):
                    types_list1.append( (t1,t2,t3,t4) )
    types_list = list(set(types_list0).union(set(types_list1)))
    types_list.sort()
    return types_list

In [5]:
#set parameters 
n = 100
T = 10000
err_rate = 0.05
sample_size = 0.1
update_rate = 0.5

gap=5

initial_beliefs = ['relatively good','relatively bad']
step=1
types_list = gen_type_list(t1=30, step=step, conditionals=[30,40,50,60,70])


# trials 0 to 2
a=0
b=2

#run and store data
start_time = pd.to_datetime('today')
print(start_time)

def runovertypes(types, n=n, T=T, gap=gap, trials=range(a, b), e=err_rate):
    type1,type2,type3,type4 = types
    df_list0 = []
    for start_belief in initial_beliefs:
        for i in trials:
            df_sub = run(n=n, T=T+1, types=types, e=e, update_rate=update_rate, start_belief=start_belief,
                             sample_size=sample_size)
            select = ((df_sub['period'] % gap) == 0) | (df_sub['period'] < 100) 
            df_sub = df_sub.loc[select]
            df_sub["trial"] = i
            df_sub["type1"] = type1
            df_sub["type2"] = type2
            df_sub["type3"] = type3
            df_sub["type4"] = type4
            df_sub["start_belief"] = start_belief
            df_list0.append(df_sub)
    df = pd.concat(df_list0)
    print(pd.to_datetime('today'), (type1,type2,type3,type4), 'done','Time used:', pd.to_datetime('today') - start_time)
    return df


pool = multiprocessing.Pool(processes=6)

try:
    df_list = pool.map(runovertypes, types_list)
except KeyboardInterrupt:
    pool.terminate()
    pool.join()

df = pd.concat(df_list)

file = f'n{n}T{T}_4types_step{step}_E{err_rate}S{sample_size}U{update_rate}IP{IP}G{gap}trial{a}_{b}.csv'
df.to_csv(file, index=False)
print('saved in', file)


2020-01-26 19:13:56.909998
2020-01-26 19:13:57.036391 (30, 2, 17, 51) done Time used: 0 days 00:00:00.127683
2020-01-26 19:13:57.084359 (30, 1, 8, 41) done Time used: 0 days 00:00:00.175702
2020-01-26 19:13:57.093887 (30, 4, 44, 22) done Time used: 0 days 00:00:00.184904
2020-01-26 19:13:57.103566 (30, 0, 0, 30) done Time used: 0 days 00:00:00.197547
2020-01-26 19:13:57.103382 (30, 3, 28, 19) done Time used: 0 days 00:00:00.194368
2020-01-26 19:13:57.111567 (30, 6, 6, 18) done Time used: 0 days 00:00:00.202913
2020-01-26 19:13:57.176956 (30, 6, 6, 28) done Time used: 0 days 00:00:00.268052
2020-01-26 19:13:57.195143 (30, 1, 8, 51) done Time used: 0 days 00:00:00.285787
2020-01-26 19:13:57.238012 (30, 2, 18, 10) done Time used: 0 days 00:00:00.328661
2020-01-26 19:13:57.271702 (30, 3, 28, 29) done Time used: 0 days 00:00:00.362266
2020-01-26 19:13:57.278205 (30, 4, 45, 1) done Time used: 0 days 00:00:00.368765
2020-01-26 19:13:57.264759 (30, 0, 0, 40) done Time used: 0 days 00:00:00.355

2020-01-26 19:13:58.938242 (30, 6, 9, 25) done Time used: 0 days 00:00:02.028825
2020-01-26 19:13:58.988125 (30, 0, 3, 47) done Time used: 0 days 00:00:02.078734
2020-01-26 19:13:59.014343 (30, 2, 21, 17) done Time used: 0 days 00:00:02.104877
2020-01-26 19:13:59.012754 (30, 1, 11, 58) done Time used: 0 days 00:00:02.104211
2020-01-26 19:13:59.015270 (30, 4, 52, 4) done Time used: 0 days 00:00:02.105766
2020-01-26 19:13:59.030846 (30, 6, 9, 35) done Time used: 0 days 00:00:02.121409
2020-01-26 19:13:59.011104 (30, 3, 32, 25) done Time used: 0 days 00:00:02.101923
2020-01-26 19:13:59.082396 (30, 0, 3, 57) done Time used: 0 days 00:00:02.172987
2020-01-26 19:13:59.103109 (30, 1, 12, 17) done Time used: 0 days 00:00:02.193707
2020-01-26 19:13:59.106642 (30, 2, 21, 27) done Time used: 0 days 00:00:02.197132
2020-01-26 19:13:59.121354 (30, 4, 52, 14) done Time used: 0 days 00:00:02.211981
2020-01-26 19:13:59.128671 (30, 6, 9, 45) done Time used: 0 days 00:00:02.219031
2020-01-26 19:13:59.12

2020-01-26 19:14:00.630323 (30, 2, 24, 24) done Time used: 0 days 00:00:03.721469
2020-01-26 19:14:00.618000 (30, 3, 36, 21) done Time used: 0 days 00:00:03.708561
2020-01-26 19:14:00.646979 (30, 0, 6, 64) done Time used: 0 days 00:00:03.737549
2020-01-26 19:14:00.671127 (30, 1, 15, 24) done Time used: 0 days 00:00:03.761719
2020-01-26 19:14:00.689246 (30, 6, 12, 52) done Time used: 0 days 00:00:03.780034
2020-01-26 19:14:00.728353 (30, 4, 64, 2) done Time used: 0 days 00:00:03.819044
2020-01-26 19:14:00.749118 (30, 2, 24, 34) done Time used: 0 days 00:00:03.839685
2020-01-26 19:14:00.749343 (30, 3, 36, 31) done Time used: 0 days 00:00:03.839980
2020-01-26 19:14:00.758619 (30, 0, 7, 23) done Time used: 0 days 00:00:03.849174
2020-01-26 19:14:00.776722 (30, 1, 15, 34) done Time used: 0 days 00:00:03.867283
2020-01-26 19:14:00.781759 (30, 6, 13, 11) done Time used: 0 days 00:00:03.872407
2020-01-26 19:14:00.814398 (30, 2, 24, 44) done Time used: 0 days 00:00:03.905002
2020-01-26 19:14:00

2020-01-26 19:14:02.475897 (30, 6, 16, 18) done Time used: 0 days 00:00:05.566487
2020-01-26 19:14:02.478336 (30, 3, 41, 26) done Time used: 0 days 00:00:05.568706
2020-01-26 19:14:02.499120 (30, 5, 2, 53) done Time used: 0 days 00:00:05.589781
2020-01-26 19:14:02.508427 (30, 2, 27, 41) done Time used: 0 days 00:00:05.599011
2020-01-26 19:14:02.511317 (30, 0, 10, 30) done Time used: 0 days 00:00:05.601867
2020-01-26 19:14:02.569767 (30, 1, 18, 41) done Time used: 0 days 00:00:05.660758
2020-01-26 19:14:02.586867 (30, 6, 16, 28) done Time used: 0 days 00:00:05.678420
2020-01-26 19:14:02.579349 (30, 3, 42, 5) done Time used: 0 days 00:00:05.671521
2020-01-26 19:14:02.604015 (30, 0, 10, 40) done Time used: 0 days 00:00:05.694654
2020-01-26 19:14:02.632590 (30, 5, 2, 63) done Time used: 0 days 00:00:05.723097
2020-01-26 19:14:02.632256 (30, 2, 28, 0) done Time used: 0 days 00:00:05.722884
2020-01-26 19:14:02.672268 (30, 1, 18, 51) done Time used: 0 days 00:00:05.762871
2020-01-26 19:14:02.

2020-01-26 19:14:04.089860 (30, 3, 47, 10) done Time used: 0 days 00:00:07.180348
2020-01-26 19:14:04.131302 (30, 1, 21, 48) done Time used: 0 days 00:00:07.221906
2020-01-26 19:14:04.137019 (30, 6, 19, 35) done Time used: 0 days 00:00:07.227579
2020-01-26 19:14:04.155266 (30, 2, 31, 37) done Time used: 0 days 00:00:07.245916
2020-01-26 19:14:04.170836 (30, 5, 6, 19) done Time used: 0 days 00:00:07.261431
2020-01-26 19:14:04.177239 (30, 3, 47, 20) done Time used: 0 days 00:00:07.267787
2020-01-26 19:14:04.182889 (30, 0, 13, 47) done Time used: 0 days 00:00:07.273495
2020-01-26 19:14:04.218549 (30, 1, 22, 7) done Time used: 0 days 00:00:07.309138
2020-01-26 19:14:04.233852 (30, 6, 19, 45) done Time used: 0 days 00:00:07.324603
2020-01-26 19:14:04.246528 (30, 2, 32, 6) done Time used: 0 days 00:00:07.337101
2020-01-26 19:14:04.261609 (30, 5, 6, 29) done Time used: 0 days 00:00:07.352166
2020-01-26 19:14:04.287194 (30, 0, 13, 57) done Time used: 0 days 00:00:07.377771
2020-01-26 19:14:04.

...

2020-01-26 19:15:50.087639 (30, 47, 23, 0) done Time used: 0 days 00:01:53.178226
2020-01-26 19:15:50.093308 (30, 31, 3, 26) done Time used: 0 days 00:01:53.183862
2020-01-26 19:15:50.123356 (30, 58, 7, 5) done Time used: 0 days 00:01:53.213958
2020-01-26 19:15:50.127386 (30, 34, 4, 2) done Time used: 0 days 00:01:53.217945
2020-01-26 19:15:50.155172 (30, 42, 1, 7) done Time used: 0 days 00:01:53.245897
2020-01-26 19:15:50.159814 (30, 37, 18, 15) done Time used: 0 days 00:01:53.250539
2020-01-26 19:15:50.182918 (30, 31, 3, 36) done Time used: 0 days 00:01:53.275014
2020-01-26 19:15:50.187296 (30, 48, 0, 2) done Time used: 0 days 00:01:53.277848
2020-01-26 19:15:50.208176 (30, 58, 8, 4) done Time used: 0 days 00:01:53.298723
2020-01-26 19:15:50.210739 (30, 34, 4, 12) done Time used: 0 days 00:01:53.301281
2020-01-26 19:15:50.226524 (30, 37, 19, 4) done Time used: 0 days 00:01:53.317078
2020-01-26 19:15:50.230005 (30, 42, 1, 17) done Time used: 0 days 00:01:53.320554
2020-01-26 19:15:50.

2020-01-26 19:15:51.556993 (30, 37, 29, 4) done Time used: 0 days 00:01:54.647543
2020-01-26 19:15:51.610755 (30, 34, 8, 28) done Time used: 0 days 00:01:54.701318
2020-01-26 19:15:51.612327 (30, 48, 6, 16) done Time used: 0 days 00:01:54.702888
2020-01-26 19:15:51.618528 (30, 59, 9, 2) done Time used: 0 days 00:01:54.709101
2020-01-26 19:15:51.639630 (30, 31, 7, 32) done Time used: 0 days 00:01:54.730820
2020-01-26 19:15:51.663733 (30, 37, 30, 3) done Time used: 0 days 00:01:54.756138
2020-01-26 19:15:51.663243 (30, 42, 6, 22) done Time used: 0 days 00:01:54.753845
2020-01-26 19:15:51.693631 (30, 59, 10, 1) done Time used: 0 days 00:01:54.784260
2020-01-26 19:15:51.691671 (30, 34, 9, 7) done Time used: 0 days 00:01:54.782081
2020-01-26 19:15:51.712471 (30, 48, 7, 5) done Time used: 0 days 00:01:54.803036
2020-01-26 19:15:51.725282 (30, 31, 8, 1) done Time used: 0 days 00:01:54.815858
2020-01-26 19:15:51.743779 (30, 37, 31, 2) done Time used: 0 days 00:01:54.834347
2020-01-26 19:15:51.

2020-01-26 19:15:52.820301 (30, 61, 2, 7) done Time used: 0 days 00:01:55.910830
2020-01-26 19:15:52.823722 (30, 34, 14, 2) done Time used: 0 days 00:01:55.914265
2020-01-26 19:15:52.848191 (30, 31, 12, 27) done Time used: 0 days 00:01:55.938762
2020-01-26 19:15:52.853469 (30, 48, 17, 5) done Time used: 0 days 00:01:55.944473
2020-01-26 19:15:52.857678 (30, 42, 13, 15) done Time used: 0 days 00:01:55.948238
2020-01-26 19:15:52.884199 (30, 38, 3, 19) done Time used: 0 days 00:01:55.974985
2020-01-26 19:15:52.889145 (30, 61, 3, 6) done Time used: 0 days 00:01:55.979662
2020-01-26 19:15:52.887960 (30, 34, 14, 12) done Time used: 0 days 00:01:55.978510
2020-01-26 19:15:52.901551 (30, 31, 13, 6) done Time used: 0 days 00:01:55.992104
2020-01-26 19:15:52.905491 (30, 48, 18, 4) done Time used: 0 days 00:01:55.996055
2020-01-26 19:15:52.910008 (30, 42, 14, 4) done Time used: 0 days 00:01:56.000573
2020-01-26 19:15:52.944127 (30, 38, 3, 29) done Time used: 0 days 00:01:56.034706
2020-01-26 19:1

2020-01-26 19:15:54.146576 (30, 63, 0, 7) done Time used: 0 days 00:01:57.238502
2020-01-26 19:15:54.140782 (30, 38, 8, 24) done Time used: 0 days 00:01:57.231375
2020-01-26 19:15:54.166519 (30, 34, 21, 5) done Time used: 0 days 00:01:57.257130
2020-01-26 19:15:54.206104 (30, 49, 4, 17) done Time used: 0 days 00:01:57.296702
2020-01-26 19:15:54.284218 (30, 42, 25, 3) done Time used: 0 days 00:01:57.374843
2020-01-26 19:15:54.348208 (30, 34, 21, 15) done Time used: 0 days 00:01:57.438800
2020-01-26 19:15:54.305229 (30, 63, 1, 6) done Time used: 0 days 00:01:57.395850
2020-01-26 19:15:54.295751 (30, 38, 9, 3) done Time used: 0 days 00:01:57.386320
2020-01-26 19:15:54.368726 (30, 49, 5, 6) done Time used: 0 days 00:01:57.459446
2020-01-26 19:15:54.397374 (30, 42, 26, 2) done Time used: 0 days 00:01:57.487997
2020-01-26 19:15:54.404935 (30, 38, 9, 13) done Time used: 0 days 00:01:57.495520
2020-01-26 19:15:54.426751 (30, 34, 22, 4) done Time used: 0 days 00:01:57.517335
2020-01-26 19:15:54

2020-01-26 19:15:56.176646 (30, 68, 2, 0) done Time used: 0 days 00:01:59.267032
2020-01-26 19:15:56.207310 (30, 69, 0, 1) done Time used: 0 days 00:01:59.297681
2020-01-26 19:15:56.230159 (30, 69, 1, 0) done Time used: 0 days 00:01:59.320513
saved in n10T5_4types_step1_E0.05S0.1U0.5IP30G5trial0_2.csv


In [6]:
# trials 2 to 5
a=2
b=5

start_time = pd.to_datetime('today')
print(start_time)

def runovertypes(types, n=n, T=T, gap=gap, trials=range(a, b), e=err_rate):
    type1,type2,type3,type4 = types
    df_list0 = []
    for start_belief in initial_beliefs:
        for i in trials:
            df_sub = run(n=n, T=T+1, types=types, e=e, update_rate=update_rate, start_belief=start_belief,
                             sample_size=sample_size)
            select = ((df_sub['period'] % gap) == 0) | (df_sub['period'] < 100) 
            df_sub = df_sub.loc[select]
            df_sub["trial"] = i
            df_sub["type1"] = type1
            df_sub["type2"] = type2
            df_sub["type3"] = type3
            df_sub["type4"] = type4
            df_sub["start_belief"] = start_belief
            df_list0.append(df_sub)
    df = pd.concat(df_list0)
    print(pd.to_datetime('today'), (type1,type2,type3,type4), 'done','Time used:', pd.to_datetime('today') - start_time)
    return df


pool = multiprocessing.Pool(processes=6)

try:
    df_list = pool.map(runovertypes, types_list)
except KeyboardInterrupt:
    pool.terminate()
    pool.join()

df = pd.concat(df_list)

file = f'n{n}T{T}_4types_step{step}_E{err_rate}S{sample_size}U{update_rate}IP{IP}G{gap}trial{a}_{b}.csv'
df.to_csv(file, index=False)
print('saved in', file)


2019-09-29 03:17:19.458012
2019-09-29 03:17:28.459400 (30, 0, 0, 30) done Time used: 0 days 00:00:09.003233
2019-09-29 03:17:28.481150 (30, 1, 8, 41) done Time used: 0 days 00:00:09.023719
2019-09-29 03:17:28.510788 (30, 6, 6, 18) done Time used: 0 days 00:00:09.053378
2019-09-29 03:17:28.564296 (30, 2, 17, 51) done Time used: 0 days 00:00:09.106931
2019-09-29 03:17:28.582575 (30, 3, 28, 19) done Time used: 0 days 00:00:09.125115
2019-09-29 03:17:28.606431 (30, 4, 44, 22) done Time used: 0 days 00:00:09.149640
2019-09-29 03:17:35.194435 (30, 0, 0, 40) done Time used: 0 days 00:00:15.736803
2019-09-29 03:17:35.200149 (30, 1, 8, 51) done Time used: 0 days 00:00:15.742490
2019-09-29 03:17:35.229188 (30, 2, 18, 10) done Time used: 0 days 00:00:15.771535
2019-09-29 03:17:35.248724 (30, 6, 6, 28) done Time used: 0 days 00:00:15.791063
2019-09-29 03:17:35.323367 (30, 4, 45, 1) done Time used: 0 days 00:00:15.865718
2019-09-29 03:17:35.356909 (30, 3, 28, 29) done Time used: 0 days 00:00:15.899

2019-09-29 03:19:21.364063 (30, 4, 51, 15) done Time used: 0 days 00:02:01.906392
2019-09-29 03:19:27.386464 (30, 1, 11, 58) done Time used: 0 days 00:02:07.928807
2019-09-29 03:19:27.656548 (30, 2, 21, 17) done Time used: 0 days 00:02:08.198907
2019-09-29 03:19:27.713616 (30, 0, 3, 47) done Time used: 0 days 00:02:08.255959
2019-09-29 03:19:28.256932 (30, 6, 9, 35) done Time used: 0 days 00:02:08.799364
2019-09-29 03:19:28.260685 (30, 3, 32, 25) done Time used: 0 days 00:02:08.803003
2019-09-29 03:19:28.497953 (30, 4, 52, 4) done Time used: 0 days 00:02:09.040278
2019-09-29 03:19:34.432233 (30, 1, 12, 17) done Time used: 0 days 00:02:14.974568
2019-09-29 03:19:34.807700 (30, 2, 21, 27) done Time used: 0 days 00:02:15.350047
2019-09-29 03:19:34.835441 (30, 0, 3, 57) done Time used: 0 days 00:02:15.377772
2019-09-29 03:19:35.356831 (30, 3, 32, 35) done Time used: 0 days 00:02:15.899169
2019-09-29 03:19:35.363892 (30, 6, 9, 45) done Time used: 0 days 00:02:15.906229
2019-09-29 03:19:35.6

2019-09-29 03:21:22.234668 (30, 6, 12, 42) done Time used: 0 days 00:04:02.777023
2019-09-29 03:21:22.264084 (30, 4, 63, 3) done Time used: 0 days 00:04:02.806417
2019-09-29 03:21:27.910706 (30, 1, 15, 24) done Time used: 0 days 00:04:08.453040
2019-09-29 03:21:28.120068 (30, 2, 24, 34) done Time used: 0 days 00:04:08.662404
2019-09-29 03:21:28.318827 (30, 0, 6, 64) done Time used: 0 days 00:04:08.861158
2019-09-29 03:21:28.891218 (30, 3, 36, 31) done Time used: 0 days 00:04:09.433548
2019-09-29 03:21:29.317997 (30, 6, 12, 52) done Time used: 0 days 00:04:09.860355
2019-09-29 03:21:29.348011 (30, 4, 64, 2) done Time used: 0 days 00:04:09.890347
2019-09-29 03:21:35.044120 (30, 1, 15, 34) done Time used: 0 days 00:04:15.586457
2019-09-29 03:21:35.273617 (30, 2, 24, 44) done Time used: 0 days 00:04:15.815951
2019-09-29 03:21:35.346329 (30, 0, 7, 23) done Time used: 0 days 00:04:15.888664
2019-09-29 03:21:35.966195 (30, 3, 37, 0) done Time used: 0 days 00:04:16.508536
2019-09-29 03:21:36.4

2019-09-29 03:23:22.215212 (30, 3, 41, 26) done Time used: 0 days 00:06:02.757545
2019-09-29 03:23:22.388913 (30, 6, 16, 8) done Time used: 0 days 00:06:02.931247
2019-09-29 03:23:23.726281 (30, 5, 2, 53) done Time used: 0 days 00:06:04.268615
2019-09-29 03:23:27.622970 (30, 1, 18, 41) done Time used: 0 days 00:06:08.165301
2019-09-29 03:23:28.015146 (30, 2, 28, 0) done Time used: 0 days 00:06:08.557482
2019-09-29 03:23:28.644703 (30, 0, 10, 30) done Time used: 0 days 00:06:09.187041
2019-09-29 03:23:29.204530 (30, 3, 42, 5) done Time used: 0 days 00:06:09.746868
2019-09-29 03:23:29.430490 (30, 6, 16, 18) done Time used: 0 days 00:06:09.972827
2019-09-29 03:23:30.891401 (30, 5, 2, 63) done Time used: 0 days 00:06:11.433736
2019-09-29 03:23:34.711483 (30, 1, 18, 51) done Time used: 0 days 00:06:15.253815
2019-09-29 03:23:35.048381 (30, 2, 28, 10) done Time used: 0 days 00:06:15.590711
2019-09-29 03:23:35.772595 (30, 0, 10, 40) done Time used: 0 days 00:06:16.314928
2019-09-29 03:23:36.3

2019-09-29 03:25:22.168290 (30, 3, 47, 10) done Time used: 0 days 00:08:02.710654
2019-09-29 03:25:22.187211 (30, 0, 13, 37) done Time used: 0 days 00:08:02.729570
2019-09-29 03:25:22.561145 (30, 6, 19, 25) done Time used: 0 days 00:08:03.103505
2019-09-29 03:25:25.486277 (30, 5, 6, 19) done Time used: 0 days 00:08:06.028631
2019-09-29 03:25:27.320933 (30, 1, 22, 7) done Time used: 0 days 00:08:07.863274
2019-09-29 03:25:28.080685 (30, 2, 32, 6) done Time used: 0 days 00:08:08.623030
2019-09-29 03:25:29.193399 (30, 3, 47, 20) done Time used: 0 days 00:08:09.735765
2019-09-29 03:25:29.235625 (30, 0, 13, 47) done Time used: 0 days 00:08:09.777970
2019-09-29 03:25:29.590326 (30, 6, 19, 35) done Time used: 0 days 00:08:10.132672
2019-09-29 03:25:32.591258 (30, 5, 6, 29) done Time used: 0 days 00:08:13.133603
2019-09-29 03:25:34.311043 (30, 1, 22, 17) done Time used: 0 days 00:08:14.853392
2019-09-29 03:25:35.076351 (30, 2, 32, 16) done Time used: 0 days 00:08:15.618702
2019-09-29 03:25:36.

...

2019-09-29 05:27:30.575327 (30, 47, 15, 8) done Time used: 0 days 02:10:11.117675
2019-09-29 05:27:31.210118 (30, 30, 28, 2) done Time used: 0 days 02:10:11.752467
2019-09-29 05:27:33.159921 (30, 33, 30, 7) done Time used: 0 days 02:10:13.702274
2019-09-29 05:27:33.949231 (30, 37, 10, 23) done Time used: 0 days 02:10:14.491580
2019-09-29 05:27:35.200171 (30, 41, 24, 5) done Time used: 0 days 02:10:15.742531
2019-09-29 05:27:35.685296 (30, 58, 0, 12) done Time used: 0 days 02:10:16.227640
2019-09-29 05:27:37.582783 (30, 47, 16, 7) done Time used: 0 days 02:10:18.125132
2019-09-29 05:27:38.169860 (30, 30, 28, 12) done Time used: 0 days 02:10:18.712218
2019-09-29 05:27:40.156689 (30, 33, 31, 6) done Time used: 0 days 02:10:20.699035
2019-09-29 05:27:40.983050 (30, 37, 11, 2) done Time used: 0 days 02:10:21.525401
2019-09-29 05:27:42.210219 (30, 41, 25, 4) done Time used: 0 days 02:10:22.752575
2019-09-29 05:27:42.735858 (30, 58, 1, 1) done Time used: 0 days 02:10:23.278206
2019-09-29 05:2

2019-09-29 05:29:28.979573 (30, 59, 0, 11) done Time used: 0 days 02:12:09.521923
2019-09-29 05:29:29.761874 (30, 48, 2, 20) done Time used: 0 days 02:12:10.304231
2019-09-29 05:29:30.082578 (30, 31, 0, 19) done Time used: 0 days 02:12:10.624928
2019-09-29 05:29:32.223529 (30, 34, 2, 14) done Time used: 0 days 02:12:12.765880
2019-09-29 05:29:33.202080 (30, 37, 17, 16) done Time used: 0 days 02:12:13.744454
2019-09-29 05:29:34.225693 (30, 42, 3, 25) done Time used: 0 days 02:12:14.768062
2019-09-29 05:29:35.973101 (30, 59, 1, 0) done Time used: 0 days 02:12:16.515451
2019-09-29 05:29:36.735056 (30, 48, 3, 9) done Time used: 0 days 02:12:17.277411
2019-09-29 05:29:37.048284 (30, 31, 0, 29) done Time used: 0 days 02:12:17.590632
2019-09-29 05:29:39.263748 (30, 34, 2, 24) done Time used: 0 days 02:12:19.806100
2019-09-29 05:29:40.167317 (30, 37, 18, 5) done Time used: 0 days 02:12:20.709688
2019-09-29 05:29:41.186988 (30, 42, 4, 4) done Time used: 0 days 02:12:21.729347
2019-09-29 05:29:4

2019-09-29 05:31:26.008283 (30, 42, 9, 9) done Time used: 0 days 02:14:06.550630
2019-09-29 05:31:28.630708 (30, 48, 11, 1) done Time used: 0 days 02:14:09.173065
2019-09-29 05:31:29.015401 (30, 60, 4, 6) done Time used: 0 days 02:14:09.557768
2019-09-29 05:31:29.543129 (30, 31, 4, 25) done Time used: 0 days 02:14:10.085491
2019-09-29 05:31:31.846185 (30, 34, 6, 20) done Time used: 0 days 02:14:12.388531
2019-09-29 05:31:32.313026 (30, 37, 28, 5) done Time used: 0 days 02:14:12.855375
2019-09-29 05:31:33.028563 (30, 42, 9, 19) done Time used: 0 days 02:14:13.570931
2019-09-29 05:31:35.627906 (30, 48, 11, 11) done Time used: 0 days 02:14:16.170259
2019-09-29 05:31:36.064191 (30, 60, 5, 5) done Time used: 0 days 02:14:16.606545
2019-09-29 05:31:36.584946 (30, 31, 4, 35) done Time used: 0 days 02:14:17.127290
2019-09-29 05:31:38.843324 (30, 34, 6, 30) done Time used: 0 days 02:14:19.385664
2019-09-29 05:31:39.291086 (30, 37, 29, 4) done Time used: 0 days 02:14:19.833457
2019-09-29 05:31:3

2019-09-29 05:33:24.654201 (30, 42, 17, 11) done Time used: 0 days 02:16:05.196554
2019-09-29 05:33:24.729221 (30, 38, 2, 20) done Time used: 0 days 02:16:05.271569
2019-09-29 05:33:27.686374 (30, 49, 1, 0) done Time used: 0 days 02:16:08.228740
2019-09-29 05:33:28.778549 (30, 31, 8, 31) done Time used: 0 days 02:16:09.320910
2019-09-29 05:33:29.495816 (30, 62, 0, 8) done Time used: 0 days 02:16:10.038164
2019-09-29 05:33:31.236720 (30, 34, 12, 4) done Time used: 0 days 02:16:11.779071
2019-09-29 05:33:31.628282 (30, 42, 18, 0) done Time used: 0 days 02:16:12.170629
2019-09-29 05:33:31.807822 (30, 38, 2, 30) done Time used: 0 days 02:16:12.350161
2019-09-29 05:33:34.660989 (30, 49, 1, 10) done Time used: 0 days 02:16:15.203366
2019-09-29 05:33:35.655995 (30, 31, 9, 0) done Time used: 0 days 02:16:16.198352
2019-09-29 05:33:36.552930 (30, 62, 1, 7) done Time used: 0 days 02:16:17.095276
2019-09-29 05:33:38.186289 (30, 34, 12, 14) done Time used: 0 days 02:16:18.728647
2019-09-29 05:33:3

2019-09-29 05:35:23.396153 (30, 34, 17, 19) done Time used: 0 days 02:18:03.938506
2019-09-29 05:35:23.494354 (30, 43, 1, 16) done Time used: 0 days 02:18:04.036723
2019-09-29 05:35:24.616551 (30, 38, 8, 4) done Time used: 0 days 02:18:05.158905
2019-09-29 05:35:26.988151 (30, 49, 9, 2) done Time used: 0 days 02:18:07.530500
2019-09-29 05:35:27.552064 (30, 31, 14, 5) done Time used: 0 days 02:18:08.094415
2019-09-29 05:35:29.799223 (30, 64, 0, 6) done Time used: 0 days 02:18:10.341584
2019-09-29 05:35:30.397361 (30, 34, 18, 8) done Time used: 0 days 02:18:10.939721
2019-09-29 05:35:30.501128 (30, 43, 1, 26) done Time used: 0 days 02:18:11.043498
2019-09-29 05:35:31.664443 (30, 38, 8, 14) done Time used: 0 days 02:18:12.206784
2019-09-29 05:35:34.079727 (30, 49, 9, 12) done Time used: 0 days 02:18:14.622086
2019-09-29 05:35:34.556012 (30, 31, 14, 15) done Time used: 0 days 02:18:15.098365
2019-09-29 05:35:36.830417 (30, 64, 1, 5) done Time used: 0 days 02:18:17.372780
2019-09-29 05:35:3